# Deployment Options: Virtual Machines vs. Kubernetes

### 1. Deploy to Kubernetes with Anyscale Operator

Anyscale offers a Kubernetes deployment option that enables you to run Ray workloads on your existing Kubernetes infrastructure. The **Anyscale Operator** is a Kubernetes-native controller that manages and orchestrates Ray clusters. 

This operator streamlines the entire lifecycle of Ray clusters by leveraging Kubernetes-native APIs and tools to handle:
- **Deployment** - Automated Ray cluster provisioning
- **Scaling** - Dynamic resource allocation based on workload demands  
- **Management** - Ongoing cluster health monitoring and maintenance

<img src="https://docs.anyscale.com/img/cloud-deployment/kubernetes.png" alt="Alt text" width="100%"/>

To install the Anyscale Operator, use Helm as you install other Kubernetes components. For detailed setup instructions and configuration options, see the [Anyscale Operator for Kubernetes documentation](https://docs.anyscale.com/administration/cloud-deployment/kubernetes#kubernetes-permissions).


##  2. Virtual Machines (VM) vs. Kubernetes (K8s)

Because Anyscale supports two infrastructure deployment types, you might wonder about the differences and which one you should use. This section explains the distinctions using AWS deployment examples, focusing on what resources Anyscale needs access to across the control layer, data layer, and workload execution.

**Key Points:**

* **Full Feature Parity**: Both VM and K8s deployments offer complete access to all Anyscale Platform features, including Workspaces, Services, Jobs, Job Queues, and advanced instance management.

* **Primarily a Preference Decision**: For most users, the choice between VM and K8s deployments comes down to organizational preference and existing infrastructure. However, the following considerations can help guide your decision.

The diagrams below illustrate the fundamental architectural differences between VM-based and Kubernetes-based Anyscale deployments:


<div style="display: flex; justify-content: space-between; align-items: flex-start; gap: 20px;">
  <div style="flex: 1; text-align: center;">
    <h3>VM-based Architecture (EC2 example)</h3>
    <img src="https://docs.anyscale.com/assets/images/aws-customer-defined-2ec2f924ecfe532b9ac8c30376c32aa4.png" alt="VM-based Anyscale Architecture" style="width: 100%; max-width: 600px;">
    <p><em>VM-based deployment with direct instance management</em></p>
  </div>
  <div style="flex: 1; text-align: center;">
    <h3>K8s-based Architecture</h3>
    <img src="https://docs.anyscale.com/img/cloud-deployment/kubernetes.png" alt="K8s-based Anyscale Architecture" style="width: 100%; max-width: 600px;">
    <p><em>Pod-based deployment with Kubernetes orchestration</em></p>
  </div>
</div>

###  2.1 Control Layer: What Anyscale Manages or Needs Access To

| Category                   | **VM-based**                                                              | **K8s-based**                                                                         |
| -------------------------- | ------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------- |
| **IAM permissions**        | • Requires a **cross-account IAM role**<br>• **Instance profiles** are used to grant access to cloud services | • **No cross-account IAM role required**—Anyscale runs as a service within your infrastructure<br>• **All cloud resources access handled** |
| **Resource management**    | • Anyscale provisions and manages EC2 instances, VPCs, security groups<br>• Full control over instance lifecycle and networking | • You manage the Kubernetes cluster; Anyscale only creates Kubernetes resources for Ray<br>• Anyscale interacts with K8s API but doesn't own cluster infrastructure |

### 2.2 Data Layer: Storage, Object Stores, and External Dependencies

| Category                               | **VM-based**                                                          | **K8s-based**                                                                              |
| ------------------------------------ | --------------------------------------------------------------- | ------------------------------------------------------------------------------------ |
| **Object Storage Access (e.g., AWS S3, GCP GCS)** | Handles via VM instance IAM roles attached to head/worker nodes.        | Handles via Pod with least-privilege policies.                      |
| **(Optional) Shared File System (e.g., AWS EFS, GCP Filestore)** | Highly recommended for workspace persistence, as well as cluster shared storage. | Highly recommended for workspace persistence, as well as cluster shared storage.       |
| **External Services Access (not in same VPC)**   | Accesses via VPC peering, private endpoints, or NAT.            | Requires K8s services (e.g., headless services, DNS) to reach data endpoints. |
| **Networking**                       | Easier control with security groups and private link endpoints. | Requires K8s ingress/egress policy control and routing.                              |

### 2.3 Workload Execution Layer: How Ray Runs on Each Backend

| Category                | **VM-based**                                                            | **K8s-based**                                                                              |
| ------------------------ | ----------------------------------------------------------------- | ------------------------------------------------------------------------------------ |
| **Cluster provisioning workflow** | User requests cluster → Anyscale assumes cross-account IAM role → Provisions VM resources → Instances auto-bootstrap and register → Automatic cleanup on teardown | User requests cluster → Anyscale Operator receives request → Provisions Ray pods via K8s scheduler → Operator manages pod lifecycle and cleanup |
| **Ray head/worker execution**      | Run directly on VM instances with full system access | Run as Kubernetes pods using container images                                       |
| **Autoscaling**          | Anyscale autoscaler controls VM nodes directly                       | • Anyscale autocaler manages pod autoscaling</br>• K8s cluster-autoscaler handles node scaling |
| **Custom Image**        | Custom AMIs not supported—use custom Docker images         | Specify custom image via patching                    |


### 2.4 When to use which

| Use Case / Requirement | **VM-based** | **K8s-based** |
| ----------------------- | ------------ | -------------- |
| **Full automation, minimal infrastructure work** | ✅ **Recommended** - Anyscale manages everything end-to-end | ❌ Requires existing K8s cluster management |
| **Fast POC (proof-of-concept) setup** | ✅ **Recommended** - Quick setup, no dependencies | ❌ More complex initial setup |
| **Fast startup time** | ✅ **Recommended** - Anyscale optimizes it. | ❌ K8s scheduler controls it |
| **Fine-grained node shape** | N/A | ✅ **Recommended** - Full Kubernetes orchestration |
| **Share compute across multiple workloads.** | N/A | ✅ **Recommended** - e.g., you plan to use schedulers like YuniKorn or queuing systems like Kueue. |

## 3. (Optional) More Kubernetes Deployments Components

- **Kubernetes Cluster Autoscaler**: Automatically scales the number of nodes in Kubernetes cluster up or down based on resource demands.

- **Load Balancer Controller: For external access to Ray applications from outside the cluster:**
  - **EKS**: Install AWS Load Balancer Controller. This creates AWS Application Load Balancers (ALB) or Network Load Balancers (NLB) that provide external access to pods.
  - **GKE**: GKE natively integrates with Google Cloud Load Balancers through built-in ingress controllers.

- **Nginx Ingress Controller**: Provides advanced HTTP routing capabilities such as:
  - Path-based routing (e.g., `/api/v1/*` to one service, `/dashboard/*` to another)
  - Host-based routing (e.g., `api.company.com` vs `dashboard.company.com`)
  - SSL/TLS termination, typically integrated with cert-manager for automatic certificate provisioning and renewal
  - Rate limiting, authentication, and request/response transformation, configurable via annotations and custom NGINX templates
  - Basic load balancing algorithms like round-robin or least connections
  
  **Note**: On EKS, Nginx Ingress Controller works behind the AWS Load Balancer Controller, which provisions an ALB or NLB to expose the ingress. 
  On GKE, it can be integrated with the Google Cloud Load Balancer by exposing the ingress via a LoadBalancer service or with GKE Ingress integration.

- **(Optional)GPU Support** The Nvidia Device Plugin is needed if you plan to run Ray jobs that require access to GPUs in your Kubernetes cluster (EKS or GKE). In Kubernetes deployment, Anyscale does not control anything outside the container, and GPU drivers are in the kernel level (and thus outside the container), and Nvidia Device Plugin sets them up.

### 4. Examples Outlook: Deploying to Your Infrastructure

**Virtual Machines (VM) Deployment**

- **[Deploy on AWS EC2](../02_02a_Deploy_to_VM_AWS_EC2/deploy_to_ec2.ipynb)**: Provisions EC2 VMs directly; Ray autoscaler manages scaling.
- **[Deploy on GCP GCE](../02_02b_Deploy_to_VM_GCP_GCE/deploy_to_GCE.ipynb)**: Provisions Compute Engine VMs; similar to EC2, but with GCP-specific IAM and networking.

**Kubernetes (K8s) Deployment**

- **[Deploy on a New AWS EKS](../02_03a_Deploy_to_K8s_New_AWS_EKS/deploy_to_a_new_EKS.ipynb)**: Deploys to a new EKS cluster; requires creating new AWS resources and installing Kubernetes controllers (autoscaler, ingress, etc.).
- **[Deploy on Existing AWS EKS](../02_03b_Deploy_to_K8s_Existing_AWS_EKS/deploy_to_an_existing_EKS.ipynb)**: Deploys to an exisitng EKS cluster; requires attaching IAM policies to node roles and installing Kubernetes controllers (autoscaler, ingress, etc.).
- **[Deploy on a New GCP GKE](../02_03c_Deploy_to_K8s_New_GCP_GKE/deploy_to_new_GKE.ipynb)**: Similar to EKS deployment, but uses GCP-native resources and IAM; requires enabling GCP APIs and configuring workload identity.
